In [5]:
import pandas as pd

# 读取数据
df = pd.read_csv(r"D:\新建文件夹\建筑数据123.csv", encoding='gbk')
print(df.head())

# 创建数据框
BA = pd.DataFrame()

   OBJECTID  height        Area  FID_ stop_name_1     BaseArea
0         1     108  970.852184    50          东赵  1348967.262
1         2       6  434.788063    50          东赵  1348967.262
2         3      36  633.525846    50          东赵  1348967.262
3         4       6  320.982952    50          东赵  1348967.262
4         5       6  299.155919    50          东赵  1348967.262


### 计算建筑密度

In [6]:
# 按FID_汇总建筑面积
building_area = df.groupby('FID_')['Area'].sum().reset_index()

# 获取每个格网的BaseArea
base_area = df.groupby('FID_')['BaseArea'].first().reset_index()

# 将两个结果合并
building_area = building_area.merge(base_area, on='FID_', how='left')

# 计算建筑密度
building_density = building_area.copy()
building_density['BD'] = building_density['Area'] / building_density['BaseArea']

# 输出结果
print(building_area)
print(building_density)

# 将汇总结果合并到BE DataFrame中
BA = building_density[['FID_', 'BD']]
BA

     FID_           Area     BaseArea
0       1  281325.451294  1351730.237
1       2  329999.934444  1352085.655
2       3  346810.602531  1352587.306
3       4  197510.954467  1352922.269
4       5   26414.783778  1331045.169
..    ...            ...          ...
158   159   13512.489092  1354113.533
159   160   63450.859554  1352423.157
160   161   67868.645362  1038537.004
161   162  129248.509229  1257246.535
162   164  135594.679515  1341047.694

[163 rows x 3 columns]
     FID_           Area     BaseArea        BD
0       1  281325.451294  1351730.237  0.208122
1       2  329999.934444  1352085.655  0.244067
2       3  346810.602531  1352587.306  0.256405
3       4  197510.954467  1352922.269  0.145988
4       5   26414.783778  1331045.169  0.019845
..    ...            ...          ...       ...
158   159   13512.489092  1354113.533  0.009979
159   160   63450.859554  1352423.157  0.046916
160   161   67868.645362  1038537.004  0.065350
161   162  129248.509229  1257246.535  0

,FID_,BD
0,1,0.208122
1,2,0.244067
2,3,0.256405
3,4,0.145988
4,5,0.019845
...,...,...
158,159,0.009979
159,160,0.046916
160,161,0.065350
161,162,0.102803


### 计算平均建筑高度

In [ ]:
# 按FID_汇总建筑高度
mean_height = df.groupby('FID_')['height'].mean().reset_index()

# 输出结果
print(mean_height)

# 转换列名
mean_height.columns = ['FID_', 'BH']

# 合并结果
BA = pd.merge(BA, mean_height, on='FID_', how='left')
BA

        Id     height
0       10   6.000000
1       18   7.333333
2       19   9.000000
3       28  36.000000
4       29  10.800000
...    ...        ...
3946  6269   6.000000
3947  6270   6.000000
3948  6271   6.545455
3949  6283   9.000000
3950  6284   8.100000

[3951 rows x 2 columns]


,Id,BD,BH
0,10,0.001686,6.000000
1,18,0.010500,7.333333
2,19,0.002888,9.000000
3,28,0.001667,36.000000
4,29,0.004135,10.800000
...,...,...,...
3946,6269,0.001518,6.000000
3947,6270,0.011880,6.000000
3948,6271,0.021303,6.545455
3949,6283,0.008850,9.000000


### 计算容积率

In [7]:
# 根据建筑高度计算层数
df['floor'] = df['height'] / 3

# 计算建筑的总建筑面积
Floor_Area = df[['FID_', 'Area', 'floor', 'BaseArea']].copy()
Floor_Area['Floor_Area'] = Floor_Area['Area'] * Floor_Area['floor']
print(Floor_Area[['FID_', 'Floor_Area']])

# 计算每个格网的总建筑面积
floor_area_ratio = Floor_Area.groupby('FID_')['Floor_Area'].sum().reset_index()

# 获取每个格网的BaseArea
base_area = Floor_Area.groupby('FID_')['BaseArea'].first().reset_index()

# 将两个结果合并
floor_area_ratio = floor_area_ratio.merge(base_area, on='FID_', how='left')

# 计算容积率
floor_area_ratio['Floor_Area'] = floor_area_ratio['Floor_Area'] / floor_area_ratio['BaseArea']

# 删除BaseArea列
floor_area_ratio = floor_area_ratio.drop(columns=['BaseArea'])

# 输出结果
print(floor_area_ratio)

# 转换列名
floor_area_ratio.columns = ['FID_', 'FAR']

# 合并结果
BA = pd.merge(BA, floor_area_ratio, on='FID_')
BA

        FID_    Floor_Area
0         50  34950.678628
1         50    869.576125
2         50   7602.310147
3         50    641.965904
4         50    598.311837
...      ...           ...
165722    25  21351.452789
165723    25      0.566958
165724    25      0.566958
165725    25      0.566958
165726    25      0.566958

[165727 rows x 2 columns]
     FID_  Floor_Area
0       1    2.428021
1       2    1.432267
2       3    1.299124
3       4    0.725936
4       5    0.201600
..    ...         ...
158   159    0.062100
159   160    1.046183
160   161    2.311739
161   162    1.701418
162   164    1.017152

[163 rows x 2 columns]


,FID_,BD,FAR
0,1,0.208122,2.428021
1,2,0.244067,1.432267
2,3,0.256405,1.299124
3,4,0.145988,0.725936
4,5,0.019845,0.201600
...,...,...,...
158,159,0.009979,0.062100
159,160,0.046916,1.046183
160,161,0.065350,2.311739
161,162,0.102803,1.701418


### 计算孔隙率

In [8]:
Calc_Volume = df[['FID_', 'Area', 'height', 'BaseArea']].copy()

# 计算建筑体积
Calc_Volume['BuildingV'] = Calc_Volume['Area'] * Calc_Volume['height']
# 对建筑体积求和
sum_volume = Calc_Volume.groupby('FID_')['BuildingV'].sum().reset_index()

# 获取格网内最大建筑高度
max_height = df.groupby('FID_')['height'].max().reset_index()
max_height.rename(columns={'height': 'max_height'}, inplace=True)

# 将两个结果合并
VoFID__Ratio = pd.merge(sum_volume, max_height, on='FID_')

# 将df中的BaseArea列合并到VoFID__Ratio中
VoFID__Ratio = pd.merge(VoFID__Ratio, df[['FID_', 'BaseArea']].drop_duplicates(), on='FID_', how='left')

# 计算格网块体积
VoFID__Ratio['GrFID_V'] = VoFID__Ratio['max_height'] * VoFID__Ratio['BaseArea']

# 计算填充率
VoFID__Ratio['Filler_Ratio'] = (VoFID__Ratio['BuildingV']) / VoFID__Ratio['GrFID_V']
# 计算孔隙率
VoFID__Ratio['VoFID__Ratio'] = 1 - VoFID__Ratio['Filler_Ratio']

# 打印结果
print(VoFID__Ratio)

# 删除多余列
VoFID__Ratio = VoFID__Ratio[['FID_', 'VoFID__Ratio']]
# 转换列名
VoFID__Ratio.columns = ['FID_', 'VR']

# 合并结果
BA = pd.merge(BA, VoFID__Ratio, on='FID_')
BA

     FID_     BuildingV  max_height     BaseArea       GrFID_V  Filler_Ratio  \
0       1  9.846088e+06         300  1351730.237  4.055191e+08      0.024280   
1       2  5.809644e+06          52  1352085.655  7.030845e+07      0.082631   
2       3  5.271536e+06         198  1352587.306  2.678123e+08      0.019684   
3       4  2.946406e+06          51  1352922.269  6.899904e+07      0.042702   
4       5  8.050159e+05          36  1331045.169  4.791763e+07      0.016800   
..    ...           ...         ...          ...           ...           ...   
158   159  2.522696e+05          36  1354113.533  4.874809e+07      0.005175   
159   160  4.244644e+06         198  1352423.157  2.677798e+08      0.015851   
160   161  7.202481e+06         360  1038537.004  3.738733e+08      0.019264   
161   162  6.417307e+06         198  1257246.535  2.489348e+08      0.025779   
162   164  4.092148e+06          72  1341047.694  9.655543e+07      0.042381   

     VoFID__Ratio  
0        0.975720  

,FID_,BD,FAR,VR
0,1,0.208122,2.428021,0.975720
1,2,0.244067,1.432267,0.917369
2,3,0.256405,1.299124,0.980316
3,4,0.145988,0.725936,0.957298
4,5,0.019845,0.201600,0.983200
...,...,...,...,...
158,159,0.009979,0.062100,0.994825
159,160,0.046916,1.046183,0.984149
160,161,0.065350,2.311739,0.980736
161,162,0.102803,1.701418,0.974221


In [9]:
# 导出结果
BA.to_csv(r"D:\新建文件夹\建筑属性.csv", index=False)